In [4]:
import radiant as rad
import numpy as np
import matplotlib.pyplot as plt


# Problem Parameters
a = 0.
b = 1.


def exact(*x):
    return np.prod(np.sin(np.asarray(x) * 2 * np.pi / (b - a)), axis=0)


def f(*x):
    return (len(x) * (2 * np.pi / (b - a)) ** 2) * exact(*x)


def g(*x):
    return np.zeros_like(x[0])

In [5]:
# Parameters
d = 2
k = 2
deltas = np.arange(0.2, 2.1, 0.1)
N = 10

# Computed Parameters
ranges = ((a, b),) * d
xc = rad.gridn(ranges, N, flat=True)

idx_funcs = [
    lambda cs: np.all([np.logical_and(c != a, c != b) for c in cs], axis=0),
    lambda cs: np.any([np.logical_or(c == a, c == b) for c in cs], axis=0),
]
operators = [
    lambda func: lambda *x: - func.laplacian(*x),
    lambda func: lambda *x: func(*x),
]

# Integration
error_integrator = rad.integrate.QuadIntegrator(ranges)

# Data collection
results = np.zeros((np.size(deltas), 3))

for i, delta in enumerate(deltas):
    # Solve for approximate solution
    phi = rad.Wendland(d, k, delta, xc)
    solver = rad.solve.CollocationSolver(phi, operators, idx_funcs)
    approx = solver.solve(f, g)
    error = rad.error(exact, approx, error_integrator, relative=False)
    cond = solver.cond()
    bw = np.max(solver.bandwidth())
    
    results[i,0] = error
    results[i, 1] = cond
    results[i, 2] = bw
    
    print("Delta:", delta)
    print("  L2 Relative Error:", error)
    print("  Condition Number:", cond)
    print("  Bandwidth:", bw)

Delta: 0.2
  L2 Relative Error: 0.616781749323488
  Condition Number: 51.0666557162626
  Bandwidth: 31
Delta: 0.30000000000000004
  L2 Relative Error: 0.920972869686792
  Condition Number: 148.90064625525542
  Bandwidth: 52
Delta: 0.4000000000000001
  L2 Relative Error: 1.0162167313081192
  Condition Number: 1175.8170966605287
  Bandwidth: 71
Delta: 0.5000000000000001
  L2 Relative Error: 1.0754772460495783
  Condition Number: 15504.1679179893
  Bandwidth: 84
Delta: 0.6000000000000001
  L2 Relative Error: 1.097087834103271
  Condition Number: 113430.90428918334
  Bandwidth: 95
Delta: 0.7000000000000002
  L2 Relative Error: 1.1058324538017565
  Condition Number: 720138.6052863131
  Bandwidth: 98
Delta: 0.8000000000000003
  L2 Relative Error: 1.1088451434458129
  Condition Number: 3073704.823493542
  Bandwidth: 99
Delta: 0.9000000000000001
  L2 Relative Error: 1.110507320298418
  Condition Number: 11417662.010478435
  Bandwidth: 99
Delta: 1.0000000000000002
  L2 Relative Error: 1.1112027

KeyboardInterrupt: 

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(12,3))

axs[0].set_xlabel(r"$\delta$")
axs[0].set_ylabel(r"$\| u - s\|_{L^{2}}$")
axs[0].set_yscale('log')
axs[0].plot(deltas, results[:, 0], 'o-')

axs[1].set_xlabel(r"$\delta$")
axs[1].set_ylabel(r"$\kappa_{2}\left(A_{L}\right)$")
axs[1].set_yscale('log')
axs[1].plot(deltas, results[:, 1], 'o-')

axs[2].set_xlabel(r"$\delta$")
axs[2].set_ylabel(r"$A_{L}$ Bandwidth")
axs[2].plot(deltas, results[:, 2], 'o-')

plt.subplots_adjust(wspace=0.5)
plt.savefig("figures/c3-kansa-poisson-deltas.pdf", bbox_inches='tight')